<a href="https://colab.research.google.com/github/alanland/colab-notebooks/blob/main/localGPT/localGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## privateGPT

https://github.com/imartinez/privateGPT


More details can be founded in [![GitHub](https://img.shields.io/github/stars/imartinez/privateGPT?style=social)](https://github.com/imartinez/privateGPT/)

In [42]:
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


## Installnation

In [43]:
#@title git clone
!git lfs install
!git clone https://github.com/imartinez/privateGPT.git
%cd privateGPT

Updated git hooks.
Git LFS initialized.
Cloning into 'privateGPT'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 286 (delta 0), reused 2 (delta 0), pack-reused 282
Receiving objects: 100% (286/286), 103.69 KiB | 5.18 MiB/s, done.
Resolving deltas: 100% (139/139), done.
/content/privateGPT/privateGPT


In [44]:
#@title checkout
#!git checkout main-local
!git pull

Already up to date.


In [45]:
#@title install dependencies

# 安装依赖
!pip install -r requirements.txt

# Install dependencies
%cd /content/privateGPT
!pip3 install -r requirements.txt

%cd /content
!rm -rf langchain
!rm -rf llama-cpp-python
!git clone https://github.com/hwchase17/langchain
!git clone https://github.com/abetlen/llama-cpp-python
%cd llama-cpp-python/vendor
!git clone https://github.com/ggerganov/llama.cpp

%cd /content

!pip3 uninstall llama-cpp-python -y
!pip3 install scikit-build
%cd /content/llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 python3 ./setup.py install

%cd /content/langchain
!pip3 uninstall langchain -y
!pip3 install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached langchain-0.0.177-py3-none-any.whl (877 kB)
  Using cached llama_cpp_python-0.1.50-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama-cpp-python 0.1.58
    Uninstalling llama-cpp-python-0.1.58:
      Successfully uninstalled llama-cpp-python-0.1.58
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.190
    Uninstalling langchain-0.0.190:
      Successfully uninstalled langchain-0.0.190
/content/privateGPT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content
Cloning into 'langchain'...
remote: Enumerating objects: 31751, done.
remote: Counting objects: 100% (14490/14490), done.
remote: Compressing objects: 100% (1560/1560), done.
remote: Total 31751 (delta 13410), reused 12966 (delta 12930), pack-reused 17261
Receiving objects: 100% (

In [46]:
#@title Install Koala 7B GGML Q4
!mkdir /content/privateGPT/models/
%cd /content/privateGPT/models/
#!wget https://huggingface.co/TheBloke/koala-7B-GGML/resolve/main/koala-7B.ggmlv3.q4_0.bin
!wget https://huggingface.co/gorborukov/koala-7b-ggml-q4_0/resolve/main/koala-7b-ggml-q4_0.bin

mkdir: cannot create directory ‘/content/privateGPT/models/’: File exists
/content/privateGPT/models
--2023-06-05 14:09:52--  https://huggingface.co/gorborukov/koala-7b-ggml-q4_0/resolve/main/koala-7b-ggml-q4_0.bin
Resolving huggingface.co (huggingface.co)... 18.172.170.44, 18.172.170.14, 18.172.170.36, ...
Connecting to huggingface.co (huggingface.co)|18.172.170.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/61/16/611642d8494b6c4086fcab6b2fed58cb350b9bda36326f536c3c61079b400c43/74cc435df7731b1d0e09fb3354d3b6dd6cc0ac5fe360bfe9f347498fed411194?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27koala-7b-ggml-q4_0.bin%3B+filename%3D%22koala-7b-ggml-q4_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1686233393&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzYxLzE2LzYxMTY0MmQ4NDk0YjZjNDA4NmZjYWI2YjJmZWQ1OGNiMzUwYjliZGEzNjMyNmY1MzZjM2M2MTA3O

In [47]:
%ls
%ls models/

koala-7b-ggml-q4_0.bin  koala-7b-ggml-q4_0.bin.1
ls: cannot access 'models/': No such file or directory


In [51]:
#@title Modify .env
!cp /content/privateGPT/example.env /content/privateGPT/.env 
!sed -i 's/n_gpu_layers=10/n_gpu_layers=500/' /content/privateGPT/.env 
!sed -i 's/MODEL_PATH=models\/koala-7B.ggml.q4_0.bin/MODEL_PATH=models\/koala-7b-ggml-q4_0.bin/' /content/privateGPT/.env 
!sed -i 's/MODEL_TYPE=GPT4All/MODEL_TYPE=LlamaCpp/' /content/privateGPT/.env 
!sed -i 's/ggml-gpt4all-j-v1.3-groovy.bin/koala-7b-ggml-q4_0.bin/' /content/privateGPT/.env 
%cat /content/privateGPT/.env

cp: cannot stat '/content/privateGPT/example.env': No such file or directory
PERSIST_DIRECTORY=db
MODEL_TYPE=LlamaCpp
MODEL_PATH=models/koala-7b-ggml-q4_0.bin
EMBEDDINGS_MODEL_NAME=all-MiniLM-L6-v2
MODEL_N_CTX=1000
TARGET_SOURCE_CHUNKS=4


In [52]:
#@title These were required before (for some filetypes), idk now
!apt install pandoc
!pip3 install unstructured

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (2.5-3build2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
#@title Download documents
%cd /content/privateGPT
%ls source_documents
!curl https://ttx-download.oss-cn-hangzhou.aliyuncs.com/temp/%E5%86%B7%E7%9F%A5%E8%AF%86.txt -o source_documents/lengzhishi.txt

/content/privateGPT
lengzhishi.txt  state_of_the_union.txt
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2477  100  2477    0     0   1084      0  0:00:02  0:00:02 --:--:--  1084


In [55]:
#@title Ingest sources
%cd  /content/privateGPT/
!python3 ingest.py

/content/privateGPT
Appending to existing vectorstore at db
Using embedded DuckDB with persistence: data will be stored in: db
Loading documents from source_documents
Loading new documents: 0it [00:00, ?it/s]
No new documents to load


In [56]:
#@title Run the chatbot
%cd /content/privateGPT
%ls models
!python3 privateGPT.py

koala-7b-ggml-q4_0.bin  koala-7b-ggml-q4_0.bin.1
ls: cannot access 'models/koala-7B.ggml.q4_0.bin': No such file or directory
/content/privateGPT
Using embedded DuckDB with persistence: data will be stored in: db
llama.cpp: loading model from models/koala-7b-ggml-q4_0.bin
terminate called after throwing an instance of 'std::runtime_error'
  what():  unexpectedly reached end of file


In [ ]:
# Clear database
!rm -rf /content/privateGPT/db

In [ ]:
# Zip the database if will reuse without recreating vectors
!zip -r /content/db.zip /content/privateGPT/db